In [1]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [2]:
# Importing self-made libaries
import sensitivity as st
import Monte_Carlo as mc
import reload_lib as rl
import life_cycle_assessment as lc
import sens_table as tab
import LCA_plots as lp
import sensitivity_case1 as c1
import sensitivity_case2 as c2


lib = [st, mc, lc, tab, lp, c1, c2]

In [3]:
from copy import deepcopy as dc

In [4]:
# Path to where the code is stored
path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care'
# Specifying the LCIA method
lcia_method = 'recipe'

ecoinevnt_paths = {'ev391apos' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets",
                   'ev391consq' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets",
                   'ev391cutoff' : r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets"}
system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
                r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

In [5]:
rl.reload_lib(lib)
variables = st.break_even_initialization(path, lcia_method)

case1_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_apos
case1_consq already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_consq
case1_cut_off already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_cut_off
case2_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_apos
case2_consq already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_consq
case2_cut_off already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_cut_off
Recipe is selected


In [ ]:
df_tot, df_scal = lc.dataframe_element_scaling(variables['case2_apos'][1])
df_tot

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))"
SUD,1.123484
MUD,0.499059


In [7]:
large_SU = False
large_REC = False
small_SU = False
small_rec = False
life_time = 500

test = False

sud = 0
mud = 0

use = 0

for lt in range(1,life_time + 1):

    for col in df_tot.columns:
        for idx, row in df_tot.iterrows():
            if 'SUD' in idx:
                sud = row[col] * lt
                
            else:
                mud = row[col] * 250 / lt
                
        if mud < sud and test == False:
            print(f"SUD = {sud}")
            print(f"MUD = {mud}")
            use = lt
            test = True

use

SUD = 12.35831989287687
MUD = 11.342260585868708


11

In [8]:
variables['case1_apos'][1]

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))"
H200 SU,"[['H200' (unit, GLO, None), 0.2476679279440376..."
H200 REC,"[['H200' (unit, GLO, None), 0.2476679279440376..."
alubox (small + dishwasher),"[['alubox (small)' (unit, GLO, None), 0.077597..."
alubox (small + wipe),"[['alubox (small)' (unit, GLO, None), 0.077597..."
H400 SU,"[['H400' (unit, GLO, None), 0.708735870176005]..."
H400 REC,"[['H400' (unit, GLO, None), 0.708735870176005]..."
alubox (large + dishwasher),"[['alubox (large)' (unit, GLO, None), 0.131076..."
alubox (large + wipe),"[['alubox (large)' (unit, GLO, None), 0.131076..."


In [9]:
df_tot, df_scal = lc.dataframe_element_scaling(variables['case1_apos'][1])
df_tot

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))"
H200 SU,0.5339
H200 REC,0.40125
alubox (small + dishwasher),0.467568
alubox (small + wipe),0.383944
H400 SU,1.358467
H400 REC,0.903042
alubox (large + dishwasher),0.851621
alubox (large + wipe),0.639172


In [10]:
col = df_tot.columns
dct = df_tot[col].to_dict()
dct_df = {}
for item in dct.values():
    for idx, val in item.items():
        dct_df[idx] = val

dct_df

scenarios = {
    'ASC : H2S' : [dct_df['alubox (small + dishwasher)'], dct_df['H200 SU']],
    'ASC : H2R' : [dct_df['alubox (small + dishwasher)'], dct_df['H200 REC']],
    'ALC : H4S' : [dct_df['alubox (large + dishwasher)'], dct_df['H400 SU']],
    'ALC : H4R' : [dct_df['alubox (large + dishwasher)'], dct_df['H400 REC']],
    'ASW : H2S' : [dct_df['alubox (small + wipe)'], dct_df['H200 SU']],
    'ASW : H2R' : [dct_df['alubox (small + wipe)'], dct_df['H200 REC']],
    'ALW : H4S' : [dct_df['alubox (large + wipe)'], dct_df['H400 SU']],
    'ALW : H4R' : [dct_df['alubox (large + wipe)'], dct_df['H400 REC']],
}

scenarios

{'ASC : H2S': [0.46756820991198483, 0.5338997152795057],
 'ASC : H2R': [0.46756820991198483, 0.4012502454513051],
 'ALC : H4S': [0.851620915095387, 1.35846749987049],
 'ALC : H4R': [0.851620915095387, 0.9030420555735845],
 'ASW : H2S': [0.3839443757626626, 0.5338997152795057],
 'ASW : H2R': [0.3839443757626626, 0.4012502454513051],
 'ALW : H4S': [0.6391719137820697, 1.35846749987049],
 'ALW : H4R': [0.6391719137820697, 0.9030420555735845]}

In [11]:
large_SU = False
large_REC = False
small_SU = False
small_rec = False
life_time = 513+314



use_dct = {}

for sc, lst in scenarios.items():
    alu = 0
    pp = 0
    test = False
    use_dct[sc] = 'Does not break even'
    for lt in range(1,life_time + 1):

        pp = lst[1] * lt
        alu = lst[0] * 513 / lt
                
        if alu < pp and test == False:
            print(f"{sc[-3:]} = {pp}, {sc[:3]} = {alu}, {lt}")
            use_dct[sc] = lt
            test = True


H2S = 11.745793736149126, ASC = 10.902840531129463, 22
H2R = 10.031256136282627, ASC = 9.594499667393928, 25
H4S = 24.452414997668818, ALC = 24.27119608021853, 18
H4R = 19.86692522261886, ALC = 19.858251338360613, 22
H2S = 10.677994305590115, ASW = 9.848173238312295, 20
H2R = 9.228755645380017, ASW = 8.563628902880257, 23
H4S = 21.73547999792784, ALW = 20.493449485637612, 16
H4R = 18.06084111147169, ALW = 16.39475958851009, 20


In [12]:
import pandas as pd
import LCA_plots as lp
import standards as s

In [13]:
513+314

827

In [14]:
file = lp.join_path(path,'results')
save_dir = s.results_folder(file, 'sensitivity', 'be')
file_path = lp.join_path(save_dir, 'break_even_data.xlsx')

sensitivity_be already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\sensitivity_be


In [15]:
df_GWP = variables['case1_apos'][1]
df_GWP

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))"
H200 SU,"[['H200' (unit, GLO, None), 0.2476679279440376..."
H200 REC,"[['H200' (unit, GLO, None), 0.2476679279440376..."
alubox (small + dishwasher),"[['alubox (small)' (unit, GLO, None), 0.077597..."
alubox (small + wipe),"[['alubox (small)' (unit, GLO, None), 0.077597..."
H400 SU,"[['H400' (unit, GLO, None), 0.708735870176005]..."
H400 REC,"[['H400' (unit, GLO, None), 0.708735870176005]..."
alubox (large + dishwasher),"[['alubox (large)' (unit, GLO, None), 0.131076..."
alubox (large + wipe),"[['alubox (large)' (unit, GLO, None), 0.131076..."


In [16]:
def df_index(key):
    if '1' in key:
        flow_leg = [
                    'H2S',
                    'H2R',
                    'ASC',
                    'ASW',
                    'H4S',
                    'H4R',
                    'ALC',
                    'ALW'
                    ]
        return flow_leg
    else:
        return ['SUD', 'MUD']
    

In [17]:
with pd.ExcelWriter(file_path) as writer:
    for key in variables.keys():
        df_GWP = variables[key][1]
        db_type = variables[key][2]
        database_name = variables[key][0]
        
        flow_legend = variables[key][4]
        columns = variables[key][-1]

        columns = lc.unique_elements_list(database_name)

        df_be, ignore = lp.process_categorizing(df_GWP, db_type, database_name, 'break even', flow_legend, columns)
        df_be.index = df_index(key)
        df_be_copy = lp.break_even_orginization(df_be, database_name)
        

        # Write each DataFrame to a different sheet
        sheet_name = f"{key}"
        df_be_copy.to_excel(writer, sheet_name=sheet_name, index=True, header=True)

In [123]:
# flow_legend, colors, save_dir, db_type, database_name = inputs

columns = lc.unique_elements_list(database_name)
case = 'break even'
df_be, ignore = lp.process_categorizing(df_GWP, db_type, database_name, case, flow_legend, columns)

amount_of_uses = 513
df_be

,Raw mat. + prod.,Disinfection,Autoclave,Recycling,Incineration,Avoided energy prod.,Avoided mat. prod.
H2S,0.247668,0.0,0.190792,0.015494,0.203083,-0.123138,0.0
H2R,0.247668,0.0,0.190792,0.067563,0.060219,-0.018471,-0.146522
ASC,0.079466,0.11211,0.296788,0.007174,0.00075,-0.000155,-0.026781
ASW,0.079466,0.057598,0.296788,0.007174,0.035186,-0.018463,-0.026781
H4S,0.708736,0.0,0.381585,0.04408,0.576358,-0.352291,0.0
H4R,0.708736,0.0,0.381585,0.191265,0.087234,-0.052844,-0.412934
ALC,0.133437,0.22422,0.534218,0.006792,0.000375,-0.00031,-0.04478
ALW,0.133437,0.057598,0.534218,0.006792,0.034811,-0.018618,-0.04478


In [127]:

if 'case1' in database_name:
    df_be_copy = lp.break_even_orginization(df_be, database_name)
    # Split index into small and large based on criteria
    small_idx = [idx for idx in df_be_copy.index if '2' in idx or 'AS' in idx]
    large_idx = [idx for idx in df_be_copy.index if idx not in small_idx]

    # Create empty DataFrames for each scenario
    scenarios = {
        'small': pd.DataFrame(0, index=small_idx, columns=df_be_copy.columns, dtype=object),
        'large': pd.DataFrame(0, index=large_idx, columns=df_be_copy.columns, dtype=object)
    }

    # Fill scenarios with data
    for sc_idx, (scenario_name, scenario_df) in enumerate(scenarios.items()):
        scenario_df.update(df_be_copy.loc[scenario_df.index])

        use_cycle, production = {}, {}

        for idx, row in scenario_df.iterrows(): 
            use, prod = 0, 0
            for col in df_be_copy.columns:
                if ('autoclave' in col.lower() or 'disinfection' in col.lower()) and 'H' not in idx:
                    use_cycle[idx] = row[col] + use
                    use += row[col]
                elif 'A' in idx:
                    # print(idx, col ,(row[col] + prod) * amount_of_uses)
                    production[idx] = (row[col] + prod) * amount_of_uses
                    prod += row[col]
                    
                else:
                    production[idx] = row[col] + prod
                    prod += row[col]
        
        # Calculate break-even values
        be_dct = {}
        for key, usage in production.items():
            be_dct[key] = [usage if u == 1 else use_cycle.get(key, usage) * u + usage
                        for u in range(1, amount_of_uses + 1)]
be_dct['ALW']

[48.998428907322975,
 50.214317219925974,
 50.82226137622747,
 51.43020553252897,
 52.03814968883047,
 52.64609384513197,
 53.25403800143347,
 53.86198215773497,
 54.46992631403647,
 55.07787047033797,
 55.68581462663947,
 56.29375878294097,
 56.90170293924247,
 57.50964709554397,
 58.11759125184547,
 58.72553540814697,
 59.33347956444847,
 59.94142372074997,
 60.549367877051466,
 61.157312033352966,
 61.76525618965447,
 62.37320034595597,
 62.98114450225747,
 63.58908865855897,
 64.19703281486046,
 64.80497697116196,
 65.41292112746348,
 66.02086528376498,
 66.62880944006648,
 67.23675359636798,
 67.84469775266948,
 68.45264190897097,
 69.06058606527247,
 69.66853022157397,
 70.27647437787547,
 70.88441853417697,
 71.49236269047847,
 72.10030684677997,
 72.70825100308147,
 73.31619515938297,
 73.92413931568447,
 74.53208347198597,
 75.14002762828747,
 75.74797178458897,
 76.35591594089047,
 76.96386009719197,
 77.57180425349347,
 78.17974840979497,
 78.78769256609647,
 79.395636722397